# Optimizing Supply Chain Using DQN

In [ ]:
import numpy as np
import tensorflow as tf
import random
from collections import deque

In [ ]:
# Supply Chain Environment
class SupplyChainEnv:
    def __init__(self):
        self.inventory = 50  # Initial inventory
        self.demand_mean = 10
        self.demand_std = 2
        self.max_inventory = 100
        self.holding_cost = 1
        self.stockout_cost = 5
        self.order_cost = 2

    def step(self, action):
        # Action is the order quantity
        self.inventory += action
        self.inventory = min(self.inventory, self.max_inventory)
        
        # Generate demand
        demand = max(0, int(np.random.normal(self.demand_mean, self.demand_std)))
        
        # Calculate costs
        if self.inventory >= demand:
            sales = demand
            holding_cost = self.holding_cost * (self.inventory - demand)
            stockout_cost = 0
        else:
            sales = self.inventory
            holding_cost = 0
            stockout_cost = self.stockout_cost * (demand - self.inventory)
        
        order_cost = self.order_cost * action
        
        # Update inventory
        self.inventory = max(0, self.inventory - demand)
        
        # Calculate reward (negative cost)
        reward = sales - (holding_cost + stockout_cost + order_cost)
        
        # State: current inventory
        state = np.array([self.inventory])
        
        return state, reward, False  # We're not using done flag in this environment

    def reset(self):
        self.inventory = 50
        return np.array([self.inventory])

In [ ]:
# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(24, activation='relu', input_shape=(self.state_size,)),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.target_model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
# Training the agent
env = SupplyChainEnv()
state_size = 1
action_size = 21  # 0 to 20 units to order
agent = DQNAgent(state_size, action_size)
batch_size = 32
episodes = 1000

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    for time in range(30):  # 30 days in a month
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    if e % 10 == 0:
        agent.update_target_model()
    print(f"Episode: {e}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2}")

In [ ]:
# Test the trained agent
test_episodes = 100
total_rewards = []

for e in range(test_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    for time in range(30):  # 30 days in a month
        action = np.argmax(agent.model.predict(state)[0])
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state
        total_reward += reward
    total_rewards.append(total_reward)

print(f"Average reward over {test_episodes} test episodes: {np.mean(total_rewards)}")